In [ ]:
IMAGE_PATH = '/content/drive/MyDrive/Dhaka-AI 2020/testset_final/set3/' #add image path

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import sys

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [ ]:
import sys

import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler

from glob import glob
from skimage import io


np.random.seed(5)

In [ ]:
types = ('*.jpg','*.jpeg','*.png','*.JPG','*.PNG') # the tuple of file types
images = []
for typ in types:
  images.extend(sorted(glob(IMAGE_PATH + typ)))
print(f' images : {len(images)} items')

 images : 510 items


In [ ]:
from random import sample

def get_ids(path_list):
  path_list = sample(path_list,len(path_list))
  id_list = [path.split('/')[-1].split('.')[0] for path in path_list]
  return np.array(id_list)

image_ids = get_ids(images)

print(image_ids)

['Pias (64)' 'Dipto_ 280' 'Dipto_ 86' 'Dipto_ 135' 'Dipto_ 44'
 'Pias (408)' 'Pias (400)' 'Dipto_ 148' 'Dipto_ 163' 'Dipto_ 180'
 'Dipto_  202' 'Dipto_ 304' 'Pias (177)' 'Dipto_ 92' 'Pias (265)'
 'Pias (430)' 'Dipto_ 109' 'Pias (220)' 'Pias (51)' 'Dipto_ 138'
 'Dipto_03' 'Dipto_  212 (1)' 'Dipto_  210' 'Dipto_ 157' 'Pias (74)'
 'Dipto_ 248' 'Pias (163)' 'Pias (31)' 'Dipto_  210 (1)' 'Dipto_  214'
 'Dipto_ 285' 'Dipto_ 287' 'Dipto_ 300' 'Pias (344)' 'Pias (222)'
 'Asraf_33' 'Dipto_ 335' 'Pias (504)' 'Pias (245)' 'Dipto_ 159'
 'Dipto_ 41' 'Pias (52)' 'Dipto_ 46' 'Dipto_ 60' 'Dipto_ 269' 'Dipto_ 48'
 'Dipto_ 40' 'Dipto_ 199' 'Pias (270)' 'Pias (58)' 'Dipto_ 325' 'Asraf_27'
 'Dipto_ 106' 'Dipto_ 246' 'Dipto_ 326' 'Pias (434)' 'Dipto_ 56'
 'Pias (357)' 'Dipto_ 331' 'Dipto_ 100' 'Dipto_ 76' 'Dipto_ 152'
 'Pias (263)' 'Dipto_ 108' 'Dipto_  219' 'Pias (528)' 'Pias (235)'
 'Dipto_ 278' 'Pias (237)' 'Dipto_ 112' 'Asraf_12' 'Pias (7)' 'Dipto_ 266'
 'Dipto_ 190' 'Dipto_  221' 'Dipto_ 75' 'Pias (72

In [ ]:
from sklearn import preprocessing
#getting the labels
marking_train = pd.read_csv("/content/drive/MyDrive/Dhaka-AI 2020/dataset/train.csv")

#A new Label dataframe storing width and height
wh_train = marking_train
wh_train['w'] = marking_train['xmax'] - marking_train['xmin']
wh_train['h'] = marking_train['ymax'] - marking_train['ymin']
wh_train = wh_train.drop(['xmax','ymax'], axis = 1)

wh_train = wh_train.rename(columns={"class" : "classname","width": "image_width", "height": "image_height", "xmin": "x", "ymin": "y"})
marking_train = wh_train
marking_train['image_id'] = marking_train['image_name']
for i in range(0,marking_train['image_name'].size) :
  marking_train['image_id'].iloc[i] = marking_train['image_name'].iloc[i].split('.')[0]
#marking_train = marking_train.drop(['image_name'], axis = 1)
le = preprocessing.LabelEncoder()
targets = le.fit_transform(marking_train['classname'].tolist())
marking_train['class_id'] = targets
marking_train = marking_train[['image_id','image_name','image_width','image_height','classname','class_id','x','y','w','h']]
marking_train = marking_train.drop(['image_name'], axis=1)
marking_train

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,image_id,image_width,image_height,classname,class_id,x,y,w,h
0,01,1200,800,bus,4,833,390,254,410
1,01,1200,800,bus,4,901,284,117,111
2,01,1200,800,bus,4,909,241,101,46
3,01,1200,800,rickshaw,13,761,413,71,127
4,01,1200,800,rickshaw,13,777,364,51,45
...,...,...,...,...,...,...,...,...,...
24363,Pias (99),854,480,car,5,376,285,97,80
24364,Pias (99),854,480,car,5,360,282,42,32
24365,Pias (99),854,480,car,5,442,275,28,26
24366,Pias (99),854,480,three wheelers (CNG),17,486,277,47,58


In [ ]:
df = marking_train

In [ ]:
df['x2'] = df['x'] + df['w'] 
df['y2'] = df['y'] + df['h']

df = df.rename(columns={'x' : 'x1', 'y' : 'y1'})
df = df.drop(columns=['class_id','w','h'])
df

,image_id,image_width,image_height,classname,x1,y1,x2,y2
0,01,1200,800,bus,833,390,1087,800
1,01,1200,800,bus,901,284,1018,395
2,01,1200,800,bus,909,241,1010,287
3,01,1200,800,rickshaw,761,413,832,540
4,01,1200,800,rickshaw,777,364,828,409
...,...,...,...,...,...,...,...,...
24363,Pias (99),854,480,car,376,285,473,365
24364,Pias (99),854,480,car,360,282,402,314
24365,Pias (99),854,480,car,442,275,470,301
24366,Pias (99),854,480,three wheelers (CNG),486,277,533,335


In [ ]:
df['classname'] = df['classname'].replace(['three_wheelers_(CNG)'],'three wheelers (CNG)')
df['classname'] = df['classname'].replace(['human_hauler'],'human hauler')
df['classname'] = df['classname'].replace(['auto_rickshaw'],'auto rickshaw')
df['classname'] = df['classname'].replace(['army_vehicle'],'army vehicle')
len(df['classname'].unique())

21

In [ ]:
class_map = dict(zip(df['classname'].unique(),list(range(1,22))))
print(df['classname'].unique())
pd.DataFrame(df['classname'].unique(),columns = ['classname']).to_csv('class.csv',index=False)

df['class'] = df['classname'].apply(lambda x : class_map[x])
# df['id'] = df['id'].astype(np.str)
df[['x1','y1','x2','y2']] = df[['x1','y1','x2','y2']].astype(np.float)
df = df.drop(index = df[df['x1']==df['x2']].index)
df.to_csv('train.csv',index=False)
# df = pd.read_csv('../input/facemask/train.csv')
df

['bus' 'rickshaw' 'motorbike' 'car' 'three wheelers (CNG)' 'pickup'
 'minivan' 'suv' 'van' 'taxi' 'truck' 'bicycle' 'policecar' 'ambulance'
 'human hauler' 'wheelbarrow' 'minibus' 'auto rickshaw' 'army vehicle'
 'scooter' 'garbagevan']


,image_id,image_width,image_height,classname,x1,y1,x2,y2,class
0,01,1200,800,bus,833.0,390.0,1087.0,800.0,1
1,01,1200,800,bus,901.0,284.0,1018.0,395.0,1
2,01,1200,800,bus,909.0,241.0,1010.0,287.0,1
3,01,1200,800,rickshaw,761.0,413.0,832.0,540.0,2
4,01,1200,800,rickshaw,777.0,364.0,828.0,409.0,2
...,...,...,...,...,...,...,...,...,...
24363,Pias (99),854,480,car,376.0,285.0,473.0,365.0,4
24364,Pias (99),854,480,car,360.0,282.0,402.0,314.0,4
24365,Pias (99),854,480,car,442.0,275.0,470.0,301.0,4
24366,Pias (99),854,480,three wheelers (CNG),486.0,277.0,533.0,335.0,5


In [ ]:
new_df = pd.DataFrame()
for i in image_ids :
  df2 = df[df['image_id']==i]
  pdList = [new_df, df2]  # List of your dataframes
  new_df = pd.concat(pdList)
  new_df = new_df.reset_index(drop=True)
new_df

,image_id,image_width,image_height,classname,x1,y1,x2,y2,class
0,Pias (64),854,480,minivan,1.0,239.0,204.0,439.0,7
1,Pias (64),854,480,minivan,344.0,260.0,372.0,290.0,7
2,Pias (64),854,480,bus,188.0,238.0,239.0,258.0,1
3,Dipto_ 280,1920,1080,rickshaw,9.0,10.0,211.0,1037.0,2
4,Dipto_ 280,1920,1080,rickshaw,861.0,147.0,988.0,348.0,2
...,...,...,...,...,...,...,...,...,...
3066,Pias (42),854,480,car,582.0,260.0,640.0,295.0,4
3067,Dipto_ 47,1920,1080,motorbike,942.0,596.0,1126.0,849.0,3
3068,Dipto_ 47,1920,1080,motorbike,853.0,556.0,892.0,616.0,3
3069,Dipto_ 47,1920,1080,motorbike,798.0,553.0,813.0,582.0,3


In [ ]:
new_df['xmin'] = new_df['x1']/new_df['image_width']
new_df['xmax'] = new_df['x2']/new_df['image_width']
new_df['ymin'] = new_df['y1']/new_df['image_height']
new_df['ymax'] = new_df['y2']/new_df['image_height']
new_df = new_df.rename(columns={'classname':'class','class':'nothing'})
new_df = new_df.drop(columns={'x1','x2','y1','y2','image_width','image_height','nothing'})
new_df = new_df[['image_id','class','xmin','ymin','xmax','ymax' ]]
new_df

,image_id,class,xmin,ymin,xmax,ymax
0,Pias (64),minivan,0.001171,0.497917,0.238876,0.914583
1,Pias (64),minivan,0.402810,0.541667,0.435597,0.604167
2,Pias (64),bus,0.220141,0.495833,0.279859,0.537500
3,Dipto_ 280,rickshaw,0.004687,0.009259,0.109896,0.960185
4,Dipto_ 280,rickshaw,0.448437,0.136111,0.514583,0.322222
...,...,...,...,...,...,...
3066,Pias (42),car,0.681499,0.541667,0.749415,0.614583
3067,Dipto_ 47,motorbike,0.490625,0.551852,0.586458,0.786111
3068,Dipto_ 47,motorbike,0.444271,0.514815,0.464583,0.570370
3069,Dipto_ 47,motorbike,0.415625,0.512037,0.423438,0.538889


In [ ]:
new_df.to_csv(IMAGE_PATH+'ground_truth_set3.csv',index=False)